In [8]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, average_precision_score
from pyod.utils.data import precision_n_scores
from pyod.models.iforest import IForest
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# Per l'uso della memoria degli algoritmi
from memory_profiler import memory_usage
# Per la metrica sul tempo di Addestramento e Inferenza
import time

In [9]:
def evaluate_metrics(y_test, y_pred, y_proba=None, digits=3):
    res = {"Accuracy": round(accuracy_score(y_test, y_pred), digits),
           "Precision": precision_score(y_test, y_pred).round(digits),
           "Recall": recall_score(y_test, y_pred).round(digits),
           "F1": f1_score(y_test, y_pred).round(digits),
           "MCC": round(matthews_corrcoef(y_test, y_pred), ndigits=digits)}
    if y_proba is not None:
        res["AUC_PR"] = average_precision_score(y_test, y_proba).round(digits)
        res["AUC_ROC"] = roc_auc_score(y_test, y_proba).round(digits)
        res["PREC_N_SCORES"] = precision_n_scores(y_test, y_proba).round(digits)
    return res


def set_seed_numpy(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [10]:
features = [
    "mean", "var", "std", "len", "duration", "len_weighted", "gaps_squared", "n_peaks",
    "smooth10_n_peaks", "smooth20_n_peaks", "var_div_duration", "var_div_len",
    "diff_peaks", "diff2_peaks", "diff_var", "diff2_var", "kurtosis", "skew",
]
SEED = 2137

In [11]:
df = pd.read_csv("data/dataset.csv", index_col="segment")

X_train, y_train = df.loc[df.train==1, features], df.loc[df.train==1, "anomaly"]
print(y_train)
X_test, y_test = df.loc[df.train==0, features], df.loc[df.train==0, "anomaly"]
X_train_nominal = df.loc[(df.anomaly==0)&(df.train==1), features]

prep = StandardScaler()
X_train_nominal2 = prep.fit_transform(X_train_nominal)
X_train2 = prep.transform(X_train)
X_test2 = prep.transform(X_test)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("X_train2", X_train2.shape)
print("X_test2", X_test2.shape)


segment
1       1
2       1
3       1
4       1
6       0
       ..
2118    0
2120    0
2121    0
2122    0
2123    1
Name: anomaly, Length: 1594, dtype: int64
X_train (1594, 18)
X_test (529, 18)
X_train2 (1594, 18)
X_test2 (529, 18)


In [12]:
set_seed_numpy(SEED) 

# Supervised Model

In [12]:
model = AdaBoostClassifier(random_state=SEED)
model.fit(X_train2, y_train)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(random_state=2137) 
 {'Accuracy': 0.934, 'Precision': 0.89, 'Recall': 0.788, 'F1': 0.836, 'MCC': 0.797, 'AUC_PR': 0.923, 'AUC_ROC': 0.962, 'PREC_N_SCORES': 0.841}


In [13]:
import xgboost as xgb

y_train_np = y_train

model = xgb.XGBClassifier (
    n_estimators=50,
    max_depth=3,
    learning_rate=0.1,
    random_state=SEED
)
model.fit(X_train, y_train)
# Previsioni e probabilità di previsione
y_predicted = model.predict(X_test)
y_predicted_score = model.predict_proba(X_test)[:, 1]  # Probabilità per la classe positiva
# Questa è la probabilità che la classificazione sia corretta

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=2137, ...) 
 {'Accuracy': 0.957, 'Precision': 0.959, 'Recall': 0.832, 'F1': 0.891, 'MCC': 0.867, 'AUC_PR': 0.961, 'AUC_ROC': 0.986, 'PREC_N_SCORES': 0.876}


In [14]:
import xgboost as xgb

y_train_np = y_train

model = xgb.XGBClassifier (
    n_estimators=50,
    max_depth=3,
    learning_rate=0.1,
    random_state=SEED
)
model.fit(X_train_scaled, y_train)
# Previsioni e probabilità di previsione
y_predicted = model.predict(X_test_scaled)
y_predicted_score = model.predict_proba(X_test_scaled)[:, 1]  # Probabilità per la classe positiva
# Questa è la probabilità che la classificazione sia corretta

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=2137, ...) 
 {'Accuracy': 0.953, 'Precision': 0.94, 'Recall': 0.832, 'F1': 0.883, 'MCC': 0.856, 'AUC_PR': 0.949, 'AUC_ROC': 0.976, 'PREC_N_SCORES': 0.867}


In [15]:
from sklearn.svm import LinearSVC

# Inizializza e addestra il modello
model = LinearSVC()
model.fit(X_train2, y_train)

# Predizione
y_test_scores = model.decision_function(X_test2)
# Previsioni e probabilità di previsione
y_predicted = model.predict(X_test2)

# Questa è la probabilità che la classificazione sia corretta
print(evaluate_metrics(y_test, y_predicted, y_predicted_score))

{'Accuracy': 0.928, 'Precision': 0.921, 'Recall': 0.726, 'F1': 0.812, 'MCC': 0.777, 'AUC_PR': 0.949, 'AUC_ROC': 0.976, 'PREC_N_SCORES': 0.867}


In [16]:
from sklearn.linear_model import LogisticRegression

# Inizializza e addestra il modello
model = LogisticRegression(max_iter=500)
model.fit(X_train2, y_train)

# Predizione
y_test_scores = model.decision_function(X_test2)
# Previsioni e probabilità di previsione
y_predicted = model.predict(X_test2)

# Questa è la probabilità che la classificazione sia corretta
print(evaluate_metrics(y_test, y_predicted, y_predicted_score))

{'Accuracy': 0.924, 'Precision': 0.92, 'Recall': 0.708, 'F1': 0.8, 'MCC': 0.764, 'AUC_PR': 0.949, 'AUC_ROC': 0.976, 'PREC_N_SCORES': 0.867}


# Unsupervised Model

MO_GAAL

In [12]:
from pyod.models.mo_gaal import MO_GAAL
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

model = MO_GAAL()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))
 # {'Accuracy': 0.896, 'Precision': 0.939, 'Recall': 0.549, 'F1': 0.693, 'MCC': 0.669, 'AUC_PR': 0.771, 'AUC_ROC': 0.849, 'PREC_N_SCORES': 0.699}

KeyboardInterrupt: 

ANO-GAAL

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "True"

# Ora importa PyOD e usa AnoGAN come prima
from pyod.models.anogan import AnoGAN
import tensorflow as tf

model = AnoGAN(verbose=1) # per stampare più cose
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))


Train iter: 1
Train iter: 2
Train iter: 3
Train iter: 4
Train iter: 5
Train iter: 6
Train iter: 7
Train iter: 8
Train iter: 9
Train iter: 10
Train iter: 11
Train iter: 12
Train iter: 13
Train iter: 14
Train iter: 15
Train iter: 16
Train iter: 17
Train iter: 18
Train iter: 19
Train iter: 20
Train iter: 21
Train iter: 22
Train iter: 23
Train iter: 24
Train iter: 25
Train iter: 26
Train iter: 27
Train iter: 28
Train iter: 29
Train iter: 30
Train iter: 31
Train iter: 32
Train iter: 33
Train iter: 34
Train iter: 35
Train iter: 36
Train iter: 37
Train iter: 38
Train iter: 39
Train iter: 40
Train iter: 41
Train iter: 42
Train iter: 43
Train iter: 44
Train iter: 45
Train iter: 46
Train iter: 47
Train iter: 48
Train iter: 49
Train iter: 50
Train iter: 51
Train iter: 52
Train iter: 53
Train iter: 54
Train iter: 55
Train iter: 56
Train iter: 57
Train iter: 58
Train iter: 59
Train iter: 60
Train iter: 61
Train iter: 62
Train iter: 63
Train iter: 64
Train iter: 65
Train iter: 66
Train iter: 67
Trai

KeyboardInterrupt: 

SO_GAAL

In [ ]:
from pyod.models.so_gaal import SO_GAAL

# Verifica le dimensioni dei dati generati
print("Dimensione X_train:", X_train.shape)
print("Dimensione y_train:", y_train.shape)
print("Dimensione X_test:", X_test.shape)
print("Dimensione y_test:", y_test.shape)

model = SO_GAAL()
model.fit(X_train2[:len(X_train2) // 500 * 500])

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

# Valutazione del modello
print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))


Dimensione X_train: (1594, 18)
Dimensione y_train: (1594,)
Dimensione X_test: (529, 18)
Dimensione y_test: (529,)
Epoch 1 of 60
Epoch 2 of 60
Epoch 3 of 60
Epoch 4 of 60
Epoch 5 of 60
Epoch 6 of 60
Epoch 7 of 60
Epoch 8 of 60
Epoch 9 of 60
Epoch 10 of 60
Epoch 11 of 60
Epoch 12 of 60
Epoch 13 of 60
Epoch 14 of 60
Epoch 15 of 60
Epoch 16 of 60
Epoch 17 of 60
Epoch 18 of 60
Epoch 19 of 60
Epoch 20 of 60
Epoch 21 of 60
Epoch 22 of 60
Epoch 23 of 60
Epoch 24 of 60
Epoch 25 of 60
Epoch 26 of 60
Epoch 27 of 60
Epoch 28 of 60
Epoch 29 of 60
Epoch 30 of 60
Epoch 31 of 60
Epoch 32 of 60
Epoch 33 of 60
Epoch 34 of 60
Epoch 35 of 60
Epoch 36 of 60
Epoch 37 of 60
Epoch 38 of 60
Epoch 39 of 60
Epoch 40 of 60
Epoch 41 of 60
Epoch 42 of 60
Epoch 43 of 60
Epoch 44 of 60
Epoch 45 of 60
Epoch 46 of 60
Epoch 47 of 60
Epoch 48 of 60
Epoch 49 of 60
Epoch 50 of 60
Epoch 51 of 60
Epoch 52 of 60
Epoch 53 of 60
Epoch 54 of 60
Epoch 55 of 60
Epoch 56 of 60
Epoch 57 of 60
Epoch 58 of 60
Epoch 59 of 60
Epoch 60 o

RF+ICCS

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Inizializza e addestra il modello
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Previsioni e probabilità di previsione
y_predicted = model.predict(X_test)
# Predizione
y_test_scores = model.predict_proba(X_test)

# Questa è la probabilità che la classificazione sia corretta
print(evaluate_metrics(y_test, y_predicted, y_predicted_score))

NameError: name 'X_train' is not defined

Linear+L2

In [ ]:
from sklearn.linear_model import RidgeClassifier

# Inizializza e addestra il modello Ridge Classifier (Linear + L2)
model = RidgeClassifier(alpha=1.0)  # 'alpha' è il parametro di regolarizzazione L2
model.fit(X_train, y_train)

# Predizione delle etichette di classe
y_predicted = model.predict(X_test)

# Ottieni le probabilità della classe positiva per AUC (si utilizza decision_function per ottenere punteggi di decisione)
y_test_scores = model.decision_function(X_test)

# Calcola e stampa le metriche
metrics = evaluate_metrics(y_test, y_predicted, y_test_scores)
print(metrics)


{'Accuracy': 0.902, 'Precision': 0.969, 'Recall': 0.558, 'F1': 0.708, 'MCC': 0.69, 'AUC_PR': 0.889, 'AUC_ROC': 0.95, 'PREC_N_SCORES': 0.814}


Isolation Forest

In [ ]:
model = IForest(random_state=SEED, contamination=.2)
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

IForest(behaviour='old', bootstrap=False, contamination=0.2, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=2137,
    verbose=0) 
 {'Accuracy': 0.701, 'Precision': 0.297, 'Recall': 0.292, 'F1': 0.295, 'MCC': 0.105, 'AUC_PR': 0.347, 'AUC_ROC': 0.635, 'PREC_N_SCORES': 0.301}


KNN

In [ ]:
from pyod.models.knn import KNN

model = KNN()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0) 
 {'Accuracy': 0.849, 'Precision': 0.78, 'Recall': 0.407, 'F1': 0.535, 'MCC': 0.489, 'AUC_PR': 0.658, 'AUC_ROC': 0.852, 'PREC_N_SCORES': 0.593}


OCSVM

In [ ]:
from pyod.models.ocsvm import OCSVM

model = OCSVM()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

OCSVM(cache_size=200, coef0=0.0, contamination=0.1, degree=3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False) 
 {'Accuracy': 0.837, 'Precision': 0.721, 'Recall': 0.389, 'F1': 0.506, 'MCC': 0.447, 'AUC_PR': 0.659, 'AUC_ROC': 0.788, 'PREC_N_SCORES': 0.655}


ABOD

In [ ]:
from pyod.models.abod import ABOD

model = ABOD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

ABOD(contamination=0.1, method='fast', n_neighbors=5) 
 {'Accuracy': 0.845, 'Precision': 0.782, 'Recall': 0.381, 'F1': 0.512, 'MCC': 0.472, 'AUC_PR': 0.644, 'AUC_ROC': 0.843, 'PREC_N_SCORES': 0.584}


INNE

In [ ]:
from pyod.models.inne import INNE

model = INNE()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

INNE(contamination=0.1, max_samples='auto', n_estimators=200,
   random_state=None) 
 {'Accuracy': 0.832, 'Precision': 0.694, 'Recall': 0.381, 'F1': 0.491, 'MCC': 0.427, 'AUC_PR': 0.636, 'AUC_ROC': 0.805, 'PREC_N_SCORES': 0.655}


ALAD

In [ ]:
from pyod.models.alad import ALAD

model = ALAD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

ALAD(activation_hidden_disc='tanh', activation_hidden_gen='tanh',
   add_disc_zz_loss=True, add_recon_loss=False, batch_size=32,
   contamination=0.1, dec_layers=[5, 10, 25], device=device(type='cpu'),
   disc_xx_layers=[25, 10, 5], disc_xz_layers=[25, 10, 5],
   disc_zz_layers=[25, 10, 5], dropout_rate=0.2, enc_layers=[25, 10, 5],
   epochs=200, lambda_recon_loss=0.1, latent_dim=2,
   learning_rate_disc=0.0001, learning_rate_gen=0.0001,
   output_activation=None, preprocessing=False,
   spectral_normalization=False, verbose=0) 
 {'Accuracy': 0.783, 'Precision': 0.485, 'Recall': 0.283, 'F1': 0.358, 'MCC': 0.25, 'AUC_PR': 0.426, 'AUC_ROC': 0.626, 'PREC_N_SCORES': 0.407}


LMDD

In [ ]:
from pyod.models.lmdd import LMDD

model = LMDD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

LMDD(contamination=0.1, dis_measure='aad', n_iter=50, random_state=None) 
 {'Accuracy': 0.822, 'Precision': 1.0, 'Recall': 0.168, 'F1': 0.288, 'MCC': 0.37, 'AUC_PR': 0.624, 'AUC_ROC': 0.765, 'PREC_N_SCORES': 0.663}


SOD

In [ ]:
from pyod.models.sod import SOD

model = SOD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

SOD(alpha=0.8, contamination=0.1, n_neighbors=20, ref_set=10) 
 {'Accuracy': 0.826, 'Precision': 0.611, 'Recall': 0.513, 'F1': 0.558, 'MCC': 0.453, 'AUC_PR': 0.621, 'AUC_ROC': 0.797, 'PREC_N_SCORES': 0.549}


COF

In [ ]:
from pyod.models.cof import COF

model = COF()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

COF(contamination=0.1, method='fast', n_neighbors=20) 
 {'Accuracy': 0.834, 'Precision': 0.667, 'Recall': 0.442, 'F1': 0.532, 'MCC': 0.449, 'AUC_PR': 0.603, 'AUC_ROC': 0.774, 'PREC_N_SCORES': 0.593}


LODA

In [ ]:
from pyod.models.loda import LODA

model = LODA()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

LODA(contamination=0.1, n_bins=10, n_random_cuts=100) 
 {'Accuracy': 0.83, 'Precision': 0.689, 'Recall': 0.372, 'F1': 0.483, 'MCC': 0.418, 'AUC_PR': 0.549, 'AUC_ROC': 0.692, 'PREC_N_SCORES': 0.522}


LUNAR

In [ ]:
from pyod.models.lunar import LUNAR

model = LUNAR()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

LUNAR(contamination=0.1, epsilon=0.1, lr=0.001, model_type='WEIGHT',
   n_epochs=200, n_neighbours=5, negative_sampling='MIXED', proportion=1.0,
   scaler=MinMaxScaler(), val_size=0.1, verbose=0, wd=0.1) 
 {'Accuracy': 0.815, 'Precision': 0.742, 'Recall': 0.204, 'F1': 0.319, 'MCC': 0.322, 'AUC_PR': 0.539, 'AUC_ROC': 0.796, 'PREC_N_SCORES': 0.451}


CBLOF

In [ ]:
from pyod.models.cblof import CBLOF

model = CBLOF()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.1, n_clusters=8, n_jobs=None, random_state=None,
   use_weights=False) 
 {'Accuracy': 0.802, 'Precision': 0.569, 'Recall': 0.292, 'F1': 0.386, 'MCC': 0.304, 'AUC_PR': 0.45, 'AUC_ROC': 0.574, 'PREC_N_SCORES': 0.372}


DIF

In [ ]:
from pyod.models.dif import DIF

model = DIF()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.predict_proba(X_test2)[:,1]

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

DIF(batch_size=1000, contamination=0.1, device=device(type='cpu'),
  hidden_activation='tanh', hidden_neurons=[500, 100], max_samples=256,
  n_ensemble=50, n_estimators=6, random_state=None, representation_dim=20,
  skip_connection=False) 
 {'Accuracy': 0.786, 'Precision': 0.5, 'Recall': 0.009, 'F1': 0.017, 'MCC': 0.043, 'AUC_PR': 0.541, 'AUC_ROC': 0.836, 'PREC_N_SCORES': 0.584}


VAE

In [ ]:
from pyod.models.vae import VAE

model = VAE()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

Training:   0%|          | 0/30 [00:00<?, ?it/s]

Training: 100%|██████████| 30/30 [00:11<00:00,  2.57it/s]


VAE(batch_norm=False, batch_size=32, beta=1.0, capacity=0.0,
  compile_mode='default', contamination=0.1,
  decoder_neuron_list=[32, 64, 128], device=device(type='cpu'),
  dropout_rate=0.2, encoder_neuron_list=[128, 64, 32], epoch_num=30,
  hidden_activation_name='relu', latent_dim=2, lr=0.001,
  optimizer_name='adam', optimizer_params={'weight_decay': 1e-05},
  output_activation_name='sigmoid', preprocessing=True, random_state=42,
  use_compile=False, verbose=1) 
 {'Accuracy': 0.794, 'Precision': 0.532, 'Recall': 0.292, 'F1': 0.377, 'MCC': 0.283, 'AUC_PR': 0.446, 'AUC_ROC': 0.687, 'PREC_N_SCORES': 0.513}


GMM

In [ ]:
from pyod.models.gmm import GMM

model = GMM()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

GMM(contamination=0.1, covariance_type='full', init_params='kmeans',
  max_iter=100, means_init=None, n_components=1, n_init=1,
  precisions_init=None, random_state=None, reg_covar=1e-06, tol=0.001,
  warm_start=False, weights_init=None) 
 {'Accuracy': 0.783, 'Precision': 0.482, 'Recall': 0.239, 'F1': 0.32, 'MCC': 0.225, 'AUC_PR': 0.426, 'AUC_ROC': 0.713, 'PREC_N_SCORES': 0.389}


DeepSVDD

In [ ]:
from pyod.models.deep_svdd import DeepSVDD

# Determina il numero di feature
n_features = X_train2.shape[1]

model = DeepSVDD(n_features=n_features)
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

Epoch 1/100, Loss: 36.17359483242035
Epoch 2/100, Loss: 36.19166633486748
Epoch 3/100, Loss: 36.2466336786747
Epoch 4/100, Loss: 36.13528761267662
Epoch 5/100, Loss: 36.165921211242676
Epoch 6/100, Loss: 36.13916572928429
Epoch 7/100, Loss: 36.189294904470444
Epoch 8/100, Loss: 36.17238187789917
Epoch 9/100, Loss: 36.2117395401001
Epoch 10/100, Loss: 36.185857594013214
Epoch 11/100, Loss: 36.13321906328201
Epoch 12/100, Loss: 36.1584706902504
Epoch 13/100, Loss: 36.17630282044411
Epoch 14/100, Loss: 36.17380636930466
Epoch 15/100, Loss: 36.25334322452545
Epoch 16/100, Loss: 36.1712027490139
Epoch 17/100, Loss: 36.12485006451607
Epoch 18/100, Loss: 36.4436274766922
Epoch 19/100, Loss: 36.22374951839447
Epoch 20/100, Loss: 36.2115415930748
Epoch 21/100, Loss: 36.16678577661514
Epoch 22/100, Loss: 36.20809951424599
Epoch 23/100, Loss: 36.228652626276016
Epoch 24/100, Loss: 36.154085248708725
Epoch 25/100, Loss: 36.138443648815155
Epoch 26/100, Loss: 36.5161928832531
Epoch 27/100, Loss: 36

PCA

In [ ]:
from pyod.models.pca import PCA

model = PCA()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

PCA(contamination=0.1, copy=True, iterated_power='auto', n_components=None,
  n_selected_components=None, random_state=None, standardization=True,
  svd_solver='auto', tol=0.0, weighted=True, whiten=False) 
 {'Accuracy': 0.779, 'Precision': 0.464, 'Recall': 0.23, 'F1': 0.308, 'MCC': 0.21, 'AUC_PR': 0.373, 'AUC_ROC': 0.612, 'PREC_N_SCORES': 0.363}


COPOD

In [ ]:
from pyod.models.copod import COPOD

model = COPOD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

COPOD(contamination=0.1, n_jobs=1) 
 {'Accuracy': 0.767, 'Precision': 0.4, 'Recall': 0.177, 'F1': 0.245, 'MCC': 0.147, 'AUC_PR': 0.328, 'AUC_ROC': 0.627, 'PREC_N_SCORES': 0.257}


SOS

In [ ]:
from pyod.models.sos import SOS

model = SOS()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

SOS(contamination=0.1, eps=1e-05, metric='euclidean', perplexity=4.5) 
 {'Accuracy': 0.758, 'Precision': 0.364, 'Recall': 0.177, 'F1': 0.238, 'MCC': 0.125, 'AUC_PR': 0.308, 'AUC_ROC': 0.524, 'PREC_N_SCORES': 0.274}


ECOD

In [ ]:
from pyod.models.ecod import ECOD

model = ECOD()
model.fit(X_train2)

y_predicted = model.predict(X_test2)
y_predicted_score = model.decision_function(X_test2)

print(model, '\n', evaluate_metrics(y_test, y_predicted, y_predicted_score))

ECOD(contamination=0.1, n_jobs=1) 
 {'Accuracy': 0.767, 'Precision': 0.396, 'Recall': 0.168, 'F1': 0.236, 'MCC': 0.14, 'AUC_PR': 0.34, 'AUC_ROC': 0.637, 'PREC_N_SCORES': 0.345}


# XGBOD

In [ ]:
from pyod.models.xgbod import XGBOD

# Inizializza e addestra XGBOD
model = XGBOD()
model.fit(X_train_scaled, y_train)

# Prevedi gli outlier nel dataset di test
y_pred = model.predict(X_test_scaled)
y_predicted_score = model.decision_function(X_test_scaled)
# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)

#n_estimators=50,
#max_depth=3,
#learning_rate=0.1,
#random_state=SEED

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:32:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=1, no...ax_features=1.0,
    max_samples='auto', n_estimators=200, n_jobs=1, random_state=0,
    verbose=0)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1,
   scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, F

#### Con metiche di Memoria e Tempo

In [ ]:
import time
from memory_profiler import memory_usage
from pyod.models.xgbod import XGBOD

# Inizializza e addestra XGBOD
model = XGBOD(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=SEED)

def train_model():
    start_time = time.time()
    mem_usage = memory_usage((model.fit, (X_train_scaled, y_train)))
    training_time = time.time() - start_time
    print(f"\n Tempo di addestramento: {training_time} secondi")
    print(f"Uso della memoria durante l'addestramento: {max(mem_usage)} MiB")
    return training_time, mem_usage

def inference_model():
    start_time = time.time()
    mem_usage_inference = memory_usage((model.predict, (X_test_scaled,)))
    inference_time = time.time() - start_time
    y_pred = model.predict(X_test_scaled)
    print(f"\n Tempo di inferenza: {inference_time} secondi")
    print(f"Uso della memoria durante l'inferenza: {max(mem_usage_inference)} MiB")
    return y_pred, inference_time, mem_usage_inference



### XGBOD più modelli unsupervised

In [ ]:
from pyod.models.xgbod import XGBOD
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.ocsvm import OCSVM

# Definizione dei modelli unsupervised
unsupervised_models = [ KNN(),
                       LOF(),
                       ABOD(),
                        OCSVM()
                    ]
# Inizializza e addestra XGBOD
model = XGBOD(estimator_list=unsupervised_models)

model.fit(X_train_scaled, y_train)

# Prevedi gli outlier nel dataset di test
y_pred = model.predict(X_test_scaled)
y_predicted_score = model.decision_function(X_test_scaled)
# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:09:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, n...3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1,
   scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True], subsample=1) {'Accuracy': 0.968, 'Precision': 0.944, 'Recall': 0.903, 'F1': 0.923, 'MCC': 0.903, 'AUC_PR': 0.974, 'AUC_ROC': 0.991, 'PREC_N_SCORES': 0.92}


#### Con Metriche di Tempo e Memoria

In [ ]:
import time
from memory_profiler import memory_usage
from pyod.models.xgbod import XGBOD

# Definizione dei modelli unsupervised
unsupervised_models = [ KNN(),
                       LOF(),
                       ABOD(),
                        OCSVM()
                    ]
# Inizializza e addestra XGBOD
model = XGBOD(estimator_list=unsupervised_models)

def train_model():
    start_time = time.time()
    mem_usage = memory_usage((model.fit, (X_train_scaled, y_train)))
    training_time = time.time() - start_time
    print(f"\n Tempo di addestramento: {training_time} secondi")
    print(f"Uso della memoria durante l'addestramento: {max(mem_usage)} MiB")
    return training_time, mem_usage

def inference_model():
    start_time = time.time()
    mem_usage_inference = memory_usage((model.predict, (X_test_scaled,)))
    inference_time = time.time() - start_time
    y_pred = model.predict(X_test_scaled)
    print(f"\n Tempo di inferenza: {inference_time} secondi")
    print(f"Uso della memoria durante l'inferenza: {max(mem_usage_inference)} MiB")
    return y_pred, inference_time, mem_usage_inference

# Addestramento del modello e monitoraggio delle metriche di efficientamento
training_time, mem_usage = train_model()

# Inferenza del modello e monitoraggio delle metriche di efficientamento
y_pred, inference_time, mem_usage_inference = inference_model()

# Calcola i punteggi di decisione
y_predicted_score = model.decision_function(X_test_scaled)

# Eseguiamo la valutazione delle metriche con le nuove metriche di efficientamento
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:13:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



 Tempo di addestramento: 2.3419463634490967 secondi
Uso della memoria durante l'addestramento: 815.8125 MiB

 Tempo di inferenza: 1.605494499206543 secondi
Uso della memoria durante l'inferenza: 815.79296875 MiB
XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, n...3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1,
   scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True], su

### XGBOD più modelli unsupervised e Parametri

In [ ]:
from pyod.models.xgbod import XGBOD
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.ocsvm import OCSVM

# Definizione dei modelli unsupervised
unsupervised_models = [ KNN(),
                       LOF(),
                       ABOD(),
                        OCSVM()
                    ]

# Inizializza e addestra XGBOD
model = XGBOD(estimator_list=unsupervised_models,
              n_estimators=100,
              max_depth=3,
              learning_rate=0.2,
              n_jobs=-1,
              random_state=SEED
            )

model.fit(X_train_scaled, y_train)

# Prevedi gli outlier nel dataset di test
y_pred = model.predict(X_test_scaled)
y_predicted_score = model.decision_function(X_test_scaled)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)
print("")
print(metrics)

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:09:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



{'Accuracy': 0.97, 'Precision': 0.945, 'Recall': 0.912, 'F1': 0.928, 'MCC': 0.909, 'AUC_PR': 0.973, 'AUC_ROC': 0.992, 'PREC_N_SCORES': 0.92}


#### Con Metriche di Tempo e Memoria

In [ ]:
import time
from memory_profiler import memory_usage
from pyod.models.xgbod import XGBOD

# Definizione dei modelli unsupervised
unsupervised_models = [ KNN(),
                       LOF(),
                       ABOD(),
                        OCSVM()
                    ]
# Inizializza e addestra XGBOD
model = XGBOD(estimator_list=unsupervised_models, n_estimators=100, max_depth=3, learning_rate=0.2, random_state=SEED)

def train_model():
    start_time = time.time()
    mem_usage = memory_usage((model.fit, (X_train_scaled, y_train)))
    training_time = time.time() - start_time
    print(f"\n Tempo di addestramento: {training_time} secondi")
    print(f"Uso della memoria durante l'addestramento: {max(mem_usage)} MiB")
    return training_time, mem_usage

def inference_model():
    start_time = time.time()
    mem_usage_inference = memory_usage((model.predict, (X_test_scaled,)))
    inference_time = time.time() - start_time
    y_pred = model.predict(X_test_scaled)
    print(f"\n Tempo di inferenza: {inference_time} secondi")
    print(f"Uso della memoria durante l'inferenza: {max(mem_usage_inference)} MiB")
    return y_pred, inference_time, mem_usage_inference

# Addestramento del modello e monitoraggio delle metriche di efficientamento
training_time, mem_usage = train_model()

# Inferenza del modello e monitoraggio delle metriche di efficientamento
y_pred, inference_time, mem_usage_inference = inference_model()

# Calcola i punteggi di decisione
y_predicted_score = model.decision_function(X_test_scaled)

# Eseguiamo la valutazione delle metriche con le nuove metriche di efficientamento
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:14:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



 Tempo di addestramento: 2.611022472381592 secondi
Uso della memoria durante l'addestramento: 816.11328125 MiB

 Tempo di inferenza: 1.9620587825775146 secondi
Uso della memoria durante l'inferenza: 816.078125 MiB
XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, n...3, gamma='auto',
   kernel='rbf', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
   verbose=False)],
   gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=2137, reg_alpha=0,
   reg_lambda=1, scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True

### Early Stopping
Termina l'esecuzione anticipatamente se per un numero prestabilito di round non migliorano più i parametri

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pyod.models.xgbod import XGBOD
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.ocsvm import OCSVM

# Definizione dei modelli unsupervised
unsupervised_models = [ KNN(),
                       LOF(),
                       ABOD(),
                        OCSVM()
                    ]

# Divisione del dataset di allenamento per avere un set di validazione
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=SEED)

# Inizializzazione del modello
model = XGBOD(estimator_list=unsupervised_models, n_estimators=50, max_depth=3, learning_rate=0.2, n_jobs=-1, random_state=SEED)

best_score = -np.inf
patience = 10       # Numero di volte che il modello cercherà di migliorarsi
patience_counter = 0
n_iterations = 100      # Numero massimo di cicli del'allenamento

for i in range(n_iterations):  # Numero massimo di iterazioni
    model.fit(X_train_sub, y_train_sub)
    
    # Predizione sul set di validazione
    y_val_pred = model.predict(X_val)
    val_score = accuracy_score(y_val, y_val_pred)
    
    # Controllo early stopping
    if val_score > best_score:
        best_score = val_score
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at iteration {i}")
            break
    model.n_estimators += 1  # Incrementa il numero di stimatori per la prossima iterazione

# Predizione sul set di test
y_pred = model.predict(X_test_scaled)
y_predicted_score = model.decision_function(X_test_scaled)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)
print("")
print(metrics)


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:09:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:09:44] WARNING:

Early stopping at iteration 12

{'Accuracy': 0.97, 'Precision': 0.971, 'Recall': 0.885, 'F1': 0.926, 'MCC': 0.909, 'AUC_PR': 0.969, 'AUC_ROC': 0.99, 'PREC_N_SCORES': 0.912}


### XGBOD con ricerca iperparametri con "grid"

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from pyod.models.xgbod import XGBOD
import numpy as np

# Definizione della griglia di parametri
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}

# Inizializza il modello
model = XGBOD()

# Randomized search con meno iterazioni e parallelizzazione
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, scoring='roc_auc', random_state=42, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# Migliori parametri trovati
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Riaddestramento del modello con i migliori parametri
model = XGBOD(**best_params)
model.fit(X_train_scaled, y_train)

# Prevedi gli outlier nel dataset di test
y_pred = model.predict(X_test_scaled)
y_predicted_score = model.decision_function(X_test_scaled)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\pyod\models\base.py:423: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:16:55] WARNING: C:\buildkite-agent\builds

Best parameters found: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:17:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=1, no...ax_features=1.0,
    max_samples='auto', n_estimators=200, n_jobs=1, random_state=0,
    verbose=0)],
   gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=50, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1,
   scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, F

### FCNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Definisci il modello FCNN
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Poiché si tratta di una classificazione binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Addestra il modello
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Prevedi gli outlier nel dataset di test
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
y_predicted_score = model.predict(X_test_scaled)

metrics = evaluate_metrics(y_test, y_pred, y_predicted_score)

# Stampa i risultati
print(model, metrics)

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8006 - loss: 0.4877 - val_accuracy: 0.8885 - val_loss: 0.2546
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9154 - loss: 0.2390 - val_accuracy: 0.9244 - val_loss: 0.1969
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9334 - loss: 0.1862 - val_accuracy: 0.9168 - val_loss: 0.1949
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9408 - loss: 0.1831 - val_accuracy: 0.9452 - val_loss: 0.1793
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9474 - loss: 0.1629 - val_accuracy: 0.9471 - val_loss: 0.1570
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9424 - loss: 0.1595 - val_accuracy: 0.9546 - val_loss: 0.1572
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9514 - loss: 0.1251 - val_accuracy: 0.9509 - val_loss: 0.1471
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9561 - loss: 0.1225 - val_accuracy: 0.9546 - val_los

# Rockad

### 2° Prova un canale -> miglioramento predizione

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

from RockadFunction import ROCKAD, NearestNeighborOCC

RANDOM_STATE = 42
STEP = 250

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef, average_precision_score

def evaluate_metrics(y_test, y_pred, y_proba=None, digits=3):
    res = {
        "Accuracy": round(accuracy_score(y_test, y_pred), digits),
        "Precision": precision_score(y_test, y_pred, average='weighted').round(digits),
        "Recall": recall_score(y_test, y_pred, average='weighted').round(digits),
        "F1": f1_score(y_test, y_pred, average='weighted').round(digits),
        "MCC": round(matthews_corrcoef(y_test, y_pred), ndigits=digits)
    }
    if y_proba is not None:
        res["AUC_PR"] = average_precision_score(y_test, y_proba, average='weighted').round(digits)
        res["AUC_ROC"] = roc_auc_score(y_test, y_proba).round(digits)
        res["PREC_N_SCORES"] = precision_n_scores(y_test, y_proba).round(digits)
    return res


# ======================= ELABORAZIONE DATI TRAINING =============================

# Lista per memorizzare i segmenti di training
X_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")
channelFix = "CADC0872"

# Itera su ogni segmento unico per il canale corrente
for segment in dfSegment[dfSegment["channel"] == channelFix]["segment"].unique():
    mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channelFix) & (dfSegment["segment"] == segment)

    # Filtra i dati in base alla maschera
    X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    # print(X_trainS.shape)
    # Suddividi in sottoliste di STEP elementi
    for i in range(0, len(X_trainS) - STEP + 1, STEP):
        sublist = X_trainS[i:i + STEP]  # Estrarre una finestra di STEP elementi
        X_train_final.append(sublist)

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train = X_train.transpose(0, 2, 1)
# print(X_train_final.shape)


# ======================= ELABORAZIONE DATI TEST =============================

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]
# Predisposizione del test set
X_test_final = []
y_test_final = []

for segment in test_data[test_data["channel"] == channelFix]["segment"].unique():

    mask = (test_data["channel"] == channelFix) & (test_data["segment"] == segment)
    X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
    y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
    
    for i in range(0, len(X_testS) - STEP + 1, STEP):
        X_test_final.append(X_testS[i:i + STEP])
        y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = X_test.transpose(0, 2, 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


y_test = np.array(y_test_final)
print("y_test: ",y_test)

# ======================= PRE-PROCESSING =============================

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================


# Inizializza e addestra il modello ROCKAD
rockad = ROCKAD(n_neighbors=5 , n_jobs=-1, n_estimators=10, n_kernels=10000, random_state=RANDOM_STATE, power_transform=False)
rockad.fit(X_train)
print("End Train")


# print("mean_train", mean_train)
# print("std_train", std_train)
# Predict anomaly scores
score_train = rockad.predict_proba(X_train)

# print("Score:", scores)

# Initialize and fit NearestNeigbor One Class Classifier

decision_func = NearestNeighborOCC().fit(score_train)
score_test = rockad.predict_proba(X_test)
print("score_test: ", score_test.shape)
print(score_test)

result = decision_func.predict(score_test)
result_binary = np.where(result == -1, 0, 1)

# result2 = knn.predict(score_test)
print("RISULTATI: ", result_binary)
#print("RISULTATI: ", result2)




metrics = evaluate_metrics(y_test, result_binary, score_test)
print("Metriche di valutazione:\n", metrics)



C:\Users\franc\AppData\Local\Temp\ipykernel_9824\51312176.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


X_train shape: (53, 1, 250)
X_test shape: (15, 1, 250)
y_test:  [0 0 1 1 1 1 0 1 1 1 1 0 1 0 0]
End Train
score_test:  (15,)
[ 59.52569593  97.91287411 104.37438939 306.12021652 117.83811987
 206.00498512  96.27331365 112.26971517  85.6070828  154.73770951
  68.09927101  68.40500052  69.69060948  70.24381485  62.94952254]
RISULTATI:  [0 0 1 1 1 0 0 1 1 1 0 0 0 1 0]
Metriche di valutazione:
 {'Accuracy': 0.733, 'Precision': 0.764, 'Recall': 0.733, 'F1': 0.736, 'MCC': 0.491, 'AUC_PR': 0.911, 'AUC_ROC': 0.833, 'PREC_N_SCORES': 0.778}


#### NORMALE

In [7]:
import numpy as np
import pandas as pd

from RockadFunction import ROCKAD, NearestNeighborOCC

features = ["channel", "segment", "value", "anomaly"]
RANDOM_STATE = 42
STEP = 250

# Lista per memorizzare i segmenti di training
X_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")
channelFix = "CADC0872"

# Itera su ogni segmento unico per il canale corrente
for segment in dfSegment[dfSegment["channel"] == channelFix]["segment"].unique():
    mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channelFix) & (dfSegment["segment"] == segment)

    # Filtra i dati in base alla maschera
    X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    # print(X_trainS.shape)
    # Suddividi in sottoliste di STEP elementi
    for i in range(0, len(X_trainS) - STEP + 1, STEP):
        sublist = X_trainS[i:i + STEP]  # Estrarre una finestra di STEP elementi
        X_train_final.append(sublist)

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
# print(X_train_final)

# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# print(X_train_final.shape)
print("X_train:", X_train.shape)

# y_train = dfSegment[dfSegment["train"] == 1]["anomaly"].values[:X_train_final.shape[0]]

# # Senza non torna perchè richiede che tutti abbiano una shape>0
# X_train_filtered, y_train_filtered = zip(*[
#     (x, y) for x, y in zip(X_train_final, y_train) if not np.any(x == 0)
# ])
# X_train_filtered = np.array(X_train_filtered)



# X_normal_train = X_train_final[y_train == 0]
#  print("Shape X_normal_train:", X_normal_train.shape)

# Inizializza e addestra il modello ROCKAD
rockad = ROCKAD(n_estimators=100, n_kernels=100, random_state=RANDOM_STATE)
rockad.fit(X_train)
print("End Train")

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]

# Predisposizione del test set
X_test_final = []
y_test_final = []

for segment in test_data[test_data["channel"] == channelFix]["segment"].unique():

    mask = (test_data["channel"] == channelFix) & (test_data["segment"] == segment)
    X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
    y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
    
    for i in range(0, len(X_testS) - STEP + 1, STEP):
        X_test_final.append(X_testS[i:i + STEP])
        y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

y_test = np.array(y_test_final)
print("y_test: ",y_test)


# Predict anomaly scores
score_train = rockad.predict_proba(X_train)
# print("Score:", scores)

# Initialize and fit NearestNeigbor One Class Classifier

decision_func = NearestNeighborOCC().fit(score_train)
score_test = rockad.predict_proba(X_test)
print("score_test: ", score_test.shape)
print(score_test)

result = decision_func.predict(score_test)
# result2 = knn.predict(score_test)
print("RISULTATI: ", result)
#print("RISULTATI: ", result2)

metrics = evaluate_metrics(y_test, result, score_test)
print("Metriche di valutazione:\n", metrics)

X_train: (53, 250, 1)
End Train


C:\Users\franc\AppData\Local\Temp\ipykernel_6284\2734640787.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


y_test:  [0 0 1 1 1 1 0 1 1 1 1 0 1 0 0]
score_test:  (15,)
[3.68062169e-07 3.68062169e-07 3.68062169e-07 3.68062169e-07
 3.68062169e-07 3.68062169e-07 3.68062169e-07 3.68062169e-07
 3.68062169e-07 3.68062169e-07 3.68062169e-07 3.68062169e-07
 3.68062169e-07 3.68062169e-07 3.68062169e-07]
RISULTATI:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Metriche di valutazione:
 {'Accuracy': 0.6, 'Precision': 0.6, 'Recall': 1.0, 'F1': 0.75, 'MCC': 0.0, 'AUC_PR': 0.6, 'AUC_ROC': 0.5, 'PREC_N_SCORES': 0.0}


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Più Canali e Miglioramento

In [ ]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from RockadFunction import ROCKAD, NearestNeighborOCC

features = ["channel", "segment", "value", "anomaly"]
RANDOM_STATE = 42
STEP = 250

# ======================= ELABORAZIONE DATI TRAINING =============================

# Lista per memorizzare i segmenti di training
X_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")

for channel in dfSegment["channel"].unique():
    # Itera su ogni segmento unico per il canale corrente
    for segment in dfSegment[dfSegment["channel"] == channel]["segment"].unique():
        mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channel) & (dfSegment["segment"] == segment)

        # Filtra i dati in base alla maschera
        X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
        # print(X_trainS.shape)
        # Suddividi in sottoliste di STEP elementi
        for i in range(0, len(X_trainS) - STEP + 1, STEP):
            sublist = X_trainS[i:i + STEP]  # Estrarre una finestra di STEP elementi
            X_train_final.append(sublist)

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
# print(X_train_final)

# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train = X_train.transpose(0, 2, 1)
print(X_train.shape)
# print("X_train_final:", X_train_final)

# ======================= ELABORAZIONE DATI TEST =============================

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]

# Predisposizione del test set
X_test_final = []
y_test_final = []

for channel in dfSegment["channel"].unique():
    for segment in test_data[test_data["channel"] == channel]["segment"].unique():

        mask = (test_data["channel"] == channel) & (test_data["segment"] == segment)
        X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
        y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
        
        for i in range(0, len(X_testS) - STEP + 1, STEP):
            X_test_final.append(X_testS[i:i + STEP])
            y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = X_test.transpose(0, 2, 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

y_test = np.array(y_test_final)
print("y_test: ",y_test.shape)

# ======================= PRE-PROCESSING =============================

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================

# Inizializza e addestra il modello ROCKAD
rockad = ROCKAD(n_estimators=10, n_kernels=1000, n_jobs=-1, random_state=RANDOM_STATE, power_transform=False)
rockad.fit(X_train)
print("End Train")

# Predict anomaly scores
score_train = rockad.predict_proba(X_train)
# print("Score:", scores)

# Initialize and fit NearestNeigbor One Class Classifier

decision_func = NearestNeighborOCC().fit(score_train)
score_test = rockad.predict_proba(X_test)
# print("score_test: ", score_test.shape)
# print(score_test)

result = decision_func.predict(score_test)
# result2 = knn.predict(score_test)
result_binary = np.where(result == -1, 0, 1)
print("RISULTATI: ", result_binary)
#print("RISULTATI: ", result2)

metrics = evaluate_metrics(y_test, result_binary, score_test)
print("Metriche di valutazione:\n", metrics)
# Senza parametri con standard scalar: {'Accuracy': 0.454, 'Precision': 0.447, 'Recall': 0.613, 'F1': 0.517, 'MCC': -0.082, 'AUC_PR': 0.407, 'AUC_ROC': 0.297, 'PREC_N_SCORES': 0.29}
# Senza standard scalar e parametri: {'Accuracy': 0.523, 'Precision': 0.5, 'Recall': 0.758, 'F1': 0.603, 'MCC': 0.075, 'AUC_PR': 0.753, 'AUC_ROC': 0.707, 'PREC_N_SCORES': 0.677}
# con parametri: {'Accuracy': 0.546, 'Precision': 0.515, 'Recall': 0.823, 'F1': 0.634, 'MCC': 0.137, 'AUC_PR': 0.757, 'AUC_ROC': 0.704, 'PREC_N_SCORES': 0.677}
# con standard scaler e parametri: {'Accuracy': 0.492, 'Precision': 0.476, 'Recall': 0.645, 'F1': 0.548, 'MCC': -0.002, 'AUC_PR': 0.407, 'AUC_ROC': 0.305, 'PREC_N_SCORES': 0.29} (10,10000)

# scaler (20, 10000): {'Accuracy': 0.531, 'Precision': 0.506, 'Recall': 0.694, 'F1': 0.585, 'MCC': 0.08, 'AUC_PR': 0.409, 'AUC_ROC': 0.305, 'PREC_N_SCORES': 0.29}
# solo param: {'Accuracy': 0.454, 'Precision': 0.447, 'Recall': 0.613, 'F1': 0.517, 'MCC': -0.082, 'AUC_PR': 0.758, 'AUC_ROC': 0.705, 'PREC_N_SCORES': 0.677}
# (30, 10000): {'Accuracy': 0.5, 'Precision': 0.481, 'Recall': 0.613, 'F1': 0.539, 'MCC': 0.01, 'AUC_PR': 0.406, 'AUC_ROC': 0.304, 'PREC_N_SCORES': 0.29}
# (35, 10000): {'Accuracy': 0.523, 'Precision': 0.5, 'Recall': 0.597, 'F1': 0.544, 'MCC': 0.053, 'AUC_PR': 0.407, 'AUC_ROC': 0.304, 'PREC_N_SCORES': 0.29}
# (40, 10000): {'Accuracy': 0.454, 'Precision': 0.443, 'Recall': 0.565, 'F1': 0.496, 'MCC': -0.084, 'AUC_PR': 0.409, 'AUC_ROC': 0.305, 'PREC_N_SCORES': 0.29}

# (10, 20000): 
# (10, 20000) + no StandardScalar:

(347, 1, 250)


C:\Users\franc\AppData\Local\Temp\ipykernel_3636\290305579.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


y_test:  (130,)


NameError: name 'StandardScaler' is not defined

## ROCKAD su NASA

In [18]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from RockadFunction import ROCKAD, NearestNeighborOCC
from NASA.nasa import NASA
from valutazione_metriche import evaluate_metrics

RANDOM_STATE = 42
STEP = 250
OUTPUT_FILE = "risultatiNASA.csv"

results_df = pd.DataFrame(columns=["Channel", "Precision", "Recall", "F1", "MCC", "AUC_ROC", "AUC_PR"])

# ======================= ELABORAZIONE DATI TRAINING =============================

# Lista per memorizzare i segmenti di training
X_train_final = []

# Uso del dataset NASA per tutti i canali
dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly")
print(dataset.data.shape)
data = dataset.data
train = []
for i in range(0, data.shape[0] - STEP +1, STEP): 
    train.append(data[i:i+STEP])

train = np.stack(train)
print("train: ", train.shape)  # Mostra le prime 5 righe dell'array


# ======================= FIT e PREDICT e SCORE =============================

# Inizializza e addestra il modello ROCKAD
rockad = ROCKAD(n_estimators=10, n_kernels=1000, n_jobs=-1, random_state=RANDOM_STATE, power_transform=False)
rockad.fit(train)
print("====================================================== End Train")

# Predict anomaly scores
score_train = rockad.predict_proba(train)
# print("Score:", scores)

dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly", train=False)
data = dataset.data
Test = []
output = []
o = np.zeros(data.shape[0])
for start,end in dataset.anomalies:
    o[start:end] = 1
for i in range(0, data.shape[0] - STEP +1, STEP): 
    Test.append(data[i:i+STEP])
    output.append(o[i:i+STEP])

output = np.stack(output)
Test = np.stack(Test)
print("TEST: ", Test.shape)  # Mostra le prime 5 righe dell'array

# Initialize and fit NearestNeigbor One Class Classifier

decision_func = NearestNeighborOCC().fit(score_train)
score_test = rockad.predict_proba(Test)
# print("score_test: ", score_test.shape)
print(score_test)

result = decision_func.predict(score_test)
# result2 = knn.predict(score_test)
result_binary = np.where(result == -1, 0, 1)
print("RISULTATI: ", result_binary)
#print("RISULTATI: ", result2)

# Scegliere se una sequenda è un anomalia o no -> 10%
threshold = 25 # -> 10%
# Conta il numero di 1 in ogni lista
counts = np.sum(output, axis=1)
output = np.where(counts >= threshold, 1, 0)

print("output: ", output)
metrics = evaluate_metrics(output, result_binary, score_test)
print("Metriche di valutazione:\n", metrics)

(2648, 25)
train:  (10, 250, 25)
====================================================== End Train
TEST:  (31, 250, 25)
[188.33394021 184.22106524 180.06022805 189.28653757 187.29862371
 175.5046242  178.16808366 192.54167875 190.22890328 185.37333139
 170.06519542 180.8084682  194.7262789  190.33104492 194.01060727
 171.50184742 182.280231   193.96076413 194.62371264 180.41964899
 185.93476758 193.34802666 187.11112602 184.39235773 179.15894819
 185.45303522 183.63233041 182.32004255 176.03040784 185.05605738
 188.93759156]
RISULTATI:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
output:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.935, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': 0.0, 'AUC_PR': 0.5, 'AUC_ROC': 0.948, 'PREC_N_SCORES': 0.5}


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from RockadFunction import ROCKAD, NearestNeighborOCC
from NASA.nasa import NASA
from valutazione_metriche import evaluate_metrics

RANDOM_STATE = 42
STEP = 250
OFFSET = 50
OUTPUT_FILE = "risultatiNASA_ROCKAD.csv"


results_df = pd.DataFrame(columns=["Channel", "Accuracy", "Precision", "Recall", "F1", "MCC", "AUC_ROC", "AUC_PR"])

# ======================= ELABORAZIONE DATI TRAINING =============================
# Itera su tutti i canali del dataset
for channel_id in NASA.channel_ids:
    # if channel_id == "D-12" or channel_id == "T-10" or channel_id == "T-9":   # Non uso questi perchè NeirestNeigthbor dato che necessita di avere più 
    #     continue
    if channel_id == "T-10":
        continue
    print(f"Processing channel: {channel_id}")

    # Lista per memorizzare i segmenti di training
    X_train_final = []

    # Uso del dataset NASA per tutti i canali
    dataset = NASA("./datasets", channel_id, mode="anomaly")
    # print(dataset.data.shape)
    data = dataset.data
    train = []
    for i in range(0, data.shape[0] - STEP +1, OFFSET): 
        train.append(data[i:i+STEP])

    train = np.stack(train)
    # print("train: ", train.shape)  # Mostra le prime 5 righe dell'array

    # ======================= FIT e PREDICT e SCORE =============================

    # Inizializza e addestra il modello ROCKAD
    rockad = ROCKAD(n_neighbors=1, n_estimators=10, n_kernels=1000, n_jobs=-1, random_state=RANDOM_STATE, power_transform=False)
    rockad.fit(train)
    print("==== End Train ====")

    # Predict anomaly scores
    score_train = rockad.predict_proba(train)
    # print("Score:", scores)

    dataset = NASA("./datasets", channel_id, mode="anomaly", train=False)
    data = dataset.data
    Test = []
    output = []
    o = np.zeros(data.shape[0])
    for start,end in dataset.anomalies:
        o[start:end] = 1
    for i in range(0, data.shape[0] - STEP +1, STEP): 
        Test.append(data[i:i+STEP])
        output.append(o[i:i+STEP])

    output = np.stack(output)
    Test = np.stack(Test)
    # print("TEST: ", Test.shape)  # Mostra le prime 5 righe dell'array

    # Initialize and fit NearestNeigbor One Class Classifier

    decision_func = NearestNeighborOCC().fit(score_train)
    
    print("Train: ", train.shape)
    print("Test: ", Test.shape)
    print("output: ", output.shape)
    score_test = rockad.predict_proba(Test)
    # print("score_test: ", score_test.shape)
    # print(score_test)

    result = decision_func.predict(score_test)
    result_binary = np.where(result == -1, 0, 1)


    # Scegliere se una sequenda è un anomalia o no -> 10%
    threshold = 1 # -> 10%
    # Conta il numero di 1 in ogni lista
    counts = np.sum(output, axis=1)
    output = np.where(counts >= threshold, 1, 0)
    print("counts: ", counts)
    print("output: ", output)
    metrics = evaluate_metrics(output, result_binary, score_test)
    print("Metriche di valutazione:\n", metrics)

    results_df = pd.concat([results_df, pd.DataFrame([{
        "Channel": channel_id,
        "Accuracy": metrics.get("Accuracy", 0),
        "Precision": metrics.get("Precision", 0),
        "Recall": metrics.get("Recall", 0),
        "MCC": metrics.get("MCC", 0),
        "AUC_PR": metrics.get("AUC_PR", 0),
        "AUC_ROC": metrics.get("AUC_ROC", 0),
        "F1": metrics.get("F1", 0),
    }])], ignore_index=True)

    print("=========================FINE CHANNEL=============================")
# ======================= SALVATAGGIO RISULTATI =============================
results_df.to_csv(OUTPUT_FILE, index=False)
print(f"Risultati salvati in {OUTPUT_FILE}")

Processing channel: A-1
==== End Train ====
Train:  (53, 250, 25)
Test:  (34, 250, 25)
output:  (34, 250)
counts:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 60. 24.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
output:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.471, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': -0.236, 'AUC_PR': 0.75, 'AUC_ROC': 0.969, 'PREC_N_SCORES': 0.5}
=========================FINE CHANNEL=============================
Processing channel: A-2
==== End Train ====


C:\Users\franc\AppData\Local\Temp\ipykernel_8292\2517294170.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{


Train:  (48, 250, 25)
Test:  (31, 250, 25)
output:  (31, 250)
counts:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 50.
 60.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
output:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.806, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': -0.101, 'AUC_PR': 0.625, 'AUC_ROC': 0.897, 'PREC_N_SCORES': 0.5}
=========================FINE CHANNEL=============================
Processing channel: A-3
==== End Train ====
Train:  (50, 250, 25)
Test:  (32, 250, 25)
output:  (32, 250)
counts:  [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0. 175.  10.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
output:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.25, 'Precision': 0.042, 'Recall': 0.5, 'F1': 0.077, 'MCC': -0.149, 'AUC_PR': 0.533, 'AUC_ROC': 0.533, 'PR

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train:  (10, 250, 25)
Test:  (18, 250, 25)
output:  (18, 250)
counts:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 50.  0.  0.  0.  0.  0.  0.]
output:  [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.944, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': 0.0, 'AUC_PR': 1.0, 'AUC_ROC': 1.0, 'PREC_N_SCORES': 1.0}
=========================FINE CHANNEL=============================
Processing channel: A-6
==== End Train ====


...

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train:  (10, 250, 55)
Test:  (6, 250, 55)
output:  (6, 250)
counts:  [ 0.  0.  0. 60.  0. 40.]
output:  [0 0 0 1 0 1]
Metriche di valutazione:
 {'Accuracy': 0.667, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': 0.0, 'AUC_PR': 0.583, 'AUC_ROC': 0.75, 'PREC_N_SCORES': 0.5}
=========================FINE CHANNEL=============================
Processing channel: T-9
==== End Train ====


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train:  (4, 250, 55)
Test:  (4, 250, 55)
output:  (4, 250)
counts:  [  0.   0.   0. 110.]
output:  [0 0 0 1]
Metriche di valutazione:
 {'Accuracy': 0.75, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': 0.0, 'AUC_PR': 1.0, 'AUC_ROC': 1.0, 'PREC_N_SCORES': 1.0}
=========================FINE CHANNEL=============================
Risultati salvati in risultatiNASA_ROCKAD.csv


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
from io import StringIO
df = pd.read_csv("risultatiNASA_ROCKAD.csv")

# Calcola la media delle colonne numeriche
column_means = df.mean(numeric_only=True)

# Stampa le medie
print("Medie delle colonne numeriche:")
print(column_means)

Medie delle colonne numeriche:
Accuracy     0.616765
Precision    0.093679
Recall       0.145272
F1           0.086543
MCC         -0.099062
AUC_ROC      0.745765
AUC_PR       0.589877
dtype: float64


---

# ROCKET

In [18]:
from sktime.transformations.panel.rocket import Rocket

import numpy as np

def detect_anomalies_with_threshold(scores, threshold):
    return (scores > threshold).astype(int)

# ======================= ELABORAZIONE DATI TRAINING =============================
STEP = 250
# Lista per memorizzare i segmenti di training
X_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")

for channel in dfSegment["channel"].unique():
    # Itera su ogni segmento unico per il canale corrente
    for segment in dfSegment[dfSegment["channel"] == channel]["segment"].unique():
    mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channelFix) & (dfSegment["segment"] == segment)

    # Filtra i dati in base alla maschera
    X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    # print(X_trainS.shape)
    # Suddividi in sottoliste di STEP elementi
    for i in range(0, len(X_trainS) - STEP + 1, STEP):
        sublist = X_trainS[i:i + STEP]  # Estrarre una finestra di STEP elementi
        X_train_final.append(sublist)

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train = X_train.transpose(0, 2, 1)
# print(X_train_final.shape)


# ======================= ELABORAZIONE DATI TEST =============================

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]
# Predisposizione del test set
X_test_final = []
y_test_final = []

for segment in test_data[test_data["channel"] == channelFix]["segment"].unique():

    mask = (test_data["channel"] == channelFix) & (test_data["segment"] == segment)
    X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
    y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
    
    for i in range(0, len(X_testS) - STEP + 1, STEP):
        X_test_final.append(X_testS[i:i + STEP])
        y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = X_test.transpose(0, 2, 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


y_test = np.array(y_test_final)
print("y_test: ",y_test)

# ======================= PRE-PROCESSING =============================

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 10000

rocket_transformer = Rocket(num_kernels = num_kernels, n_jobs=-1)


# Applica i kernel alle serie temporali
features_train = rocket_transformer.fit_transform(X_train)
features_test = rocket_transformer.transform(X_test)
print("features_train: ", features_train.shape)
print("features_test: ", features_test.shape)
# Sintesi delle caratteristiche per esempio
anomaly_scores_train = np.mean(features_train, axis=1)  
anomaly_scores_test = np.mean(features_test, axis=1)  

# Rilevamento delle anomalie
threshold = np.percentile(anomaly_scores_train , 95)
anomaly_labels_train = detect_anomalies_with_threshold(anomaly_scores_train , threshold)
anomaly_labels_test = detect_anomalies_with_threshold(anomaly_scores_test , threshold)

print(np.unique(anomaly_labels_test, return_counts=True))

# Visualizzazione dei risultati
print("Anomalie rilevate nel training set:", anomaly_labels_train)
print("Anomalie rilevate nel test set:", anomaly_labels_test)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, anomaly_labels_test)
print("Metriche di valutazione sul test set:\n", metrics)

C:\Users\franc\AppData\Local\Temp\ipykernel_8292\3020148067.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


X_train shape: (53, 1, 250)
X_test shape: (15, 1, 250)
y_test:  [0 0 1 1 1 1 0 1 1 1 1 0 1 0 0]
features_train:  (53, 20000)
features_test:  (15, 20000)
(array([0, 1]), array([14,  1], dtype=int64))
Anomalie rilevate nel training set: 0     1
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    1
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    1
49    0
50    0
51    0
52    0
dtype: int32
Anomalie rilevate nel test set: 0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int32
Metriche di valutazione sul test set:
 {'Accuracy': 0.467, 'Precision': 1.0, 'Recall': 0.111, 'F1': 0.2, 'MCC': 0.218}


## Unsupervised con Threshold

In [13]:
import numpy as np
from sktime.transformations.panel.rocket import Rocket

def detect_anomalies_with_threshold(scores, threshold):
    return (scores > threshold).astype(int)

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 10000
kernels = generate_kernels(input_length, num_kernels)

# Applica i kernel alle serie temporali
features_train = apply_kernels(X_train2, kernels)
features_test = apply_kernels(X_test2, kernels)

print(X_train2.shape)
print(X_test2.shape)

# Sintesi delle caratteristiche per esempio
anomaly_scores_train = np.mean(features_train, axis=1)  
anomaly_scores_test = np.mean(features_test, axis=1)  

# Rilevamento delle anomalie
threshold = np.percentile(anomaly_scores_train , 95)
anomaly_labels_train = detect_anomalies_with_threshold(anomaly_scores_train , threshold)
anomaly_labels_test = detect_anomalies_with_threshold(anomaly_scores_test , threshold)

# Visualizzazione dei risultati
print("Anomalie rilevate nel training set:", anomaly_labels_train.shape)
print("Anomalie rilevate nel test set:", anomaly_labels_test.shape)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, anomaly_labels_test)
print("Metriche di valutazione sul test set:\n", metrics)
# {'Accuracy': 0.832, 'Precision': 0.962, 'Recall': 0.221, 'F1': 0.36, 'MCC': 0.415, 'AUC_PR': 0.726, 'AUC_ROC': 0.772, 'PREC_N_SCORES': 0.646}

(1594, 18)
(529, 18)
Anomalie rilevate nel training set: (1594,)
Anomalie rilevate nel test set: (529,)
Metriche di valutazione sul test set:
 {'Accuracy': 0.832, 'Precision': 0.962, 'Recall': 0.221, 'F1': 0.36, 'MCC': 0.415}


In [10]:
import numpy as np
from rocket_functions import generate_kernels, apply_kernels

def detect_anomalies_with_threshold(scores, threshold):
    return (scores > threshold).astype(int)

# ======================= ELABORAZIONE DATI TRAINING =============================
STEP = 250
# Lista per memorizzare i segmenti di training
X_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")
channelFix = "CADC0872"

# Itera su ogni segmento unico per il canale corrente
for segment in dfSegment[dfSegment["channel"] == channelFix]["segment"].unique():
    mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channelFix) & (dfSegment["segment"] == segment)

    # Filtra i dati in base alla maschera
    X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    # print(X_trainS.shape)
    # Suddividi in sottoliste di STEP elementi
    for i in range(0, len(X_trainS) - STEP + 1, STEP):
        sublist = X_trainS[i:i + STEP]  # Estrarre una finestra di STEP elementi
        X_train_final.append(sublist)

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train = X_train.transpose(0, 2, 1)
# print(X_train_final.shape)


# ======================= ELABORAZIONE DATI TEST =============================

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]
# Predisposizione del test set
X_test_final = []
y_test_final = []

for segment in test_data[test_data["channel"] == channelFix]["segment"].unique():

    mask = (test_data["channel"] == channelFix) & (test_data["segment"] == segment)
    X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
    y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
    
    for i in range(0, len(X_testS) - STEP + 1, STEP):
        X_test_final.append(X_testS[i:i + STEP])
        y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = X_test.transpose(0, 2, 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


y_test = np.array(y_test_final)
print("y_test: ",y_test)

# ======================= PRE-PROCESSING =============================

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================

X_train = X_train.reshape(X_train.shape[0], -1)  # Appiattimento in 2D
X_test = X_test.reshape(X_test.shape[0], -1)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 10000
kernels = generate_kernels(input_length, num_kernels)

# Applica i kernel alle serie temporali
features_train = apply_kernels(X_train, kernels)
features_test = apply_kernels(X_test, kernels)

# Sintesi delle caratteristiche per esempio
anomaly_scores_train = np.mean(features_train, axis=1)  
anomaly_scores_test = np.mean(features_test, axis=1)  

# Rilevamento delle anomalie
threshold = np.percentile(anomaly_scores_train , 95)
anomaly_labels_train = detect_anomalies_with_threshold(anomaly_scores_train , threshold)
anomaly_labels_test = detect_anomalies_with_threshold(anomaly_scores_test , threshold)

# Visualizzazione dei risultati
print("Anomalie rilevate nel training set:", anomaly_labels_train)
print("Anomalie rilevate nel test set:", anomaly_labels_test)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, anomaly_labels_test)
print("Metriche di valutazione sul test set:\n", metrics)

C:\Users\franc\AppData\Local\Temp\ipykernel_23708\2536707538.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


X_train shape: (53, 1, 250)
X_test shape: (15, 1, 250)
y_test:  [0 0 1 1 1 1 0 1 1 1 1 0 1 0 0]
X_train shape: (53, 250)
X_test shape: (15, 250)
Anomalie rilevate nel training set: [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Anomalie rilevate nel test set: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Metriche di valutazione sul test set:
 {'Accuracy': 0.467, 'Precision': 1.0, 'Recall': 0.111, 'F1': 0.2, 'MCC': 0.218}


### KNN ( UNSUPERVISED )

In [14]:
from pyod.models.knn import KNN
from sktime.transformations.panel.rocket import Rocket

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 1000
kernels = generate_kernels(input_length, num_kernels)

# Applica i kernel alle serie temporali
features_train = apply_kernels(X_train2, kernels)
features_test = apply_kernels(X_test2, kernels)


# Addestramento del modello supervisionato
model = KNN()
model.fit(features_train)

# Predizione delle anomalie nei dati di test
y_pred = model.predict(features_test)
y_proba = model.decision_function(features_test)

# Visualizzazione dei risultati
print("Predizioni nel test set:", y_pred.shape)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_proba)
print("Metriche di valutazione:\n", metrics)
# {'Accuracy': 0.845, 'Precision': 0.763, 'Recall': 0.398, 'F1': 0.523, 'MCC': 0.475, 'AUC_PR': 0.619, 'AUC_ROC': 0.811, 'PREC_N_SCORES': 0.54}

Predizioni nel test set: (529,)
Metriche di valutazione:
 {'Accuracy': 0.845, 'Precision': 0.754, 'Recall': 0.407, 'F1': 0.529, 'MCC': 0.476, 'AUC_PR': 0.611, 'AUC_ROC': 0.804, 'PREC_N_SCORES': 0.531}


## Rilevamento di anomalie ROCKET SUPERVISED
Utilizzo di vari algoritmi unsupervised e non con kernel ROCKET

### Regressione Logistica -> Classificatore lineare ( SUPERVISED )

In [ ]:
from sklearn.linear_model import LogisticRegression

from rocket_functions import generate_kernels, apply_kernels

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 1000
kernels = generate_kernels(input_length, num_kernels)

# Applica i kernel alle serie temporali
features_train = apply_kernels(X_train2, kernels)
features_test = apply_kernels(X_test2, kernels)


# Addestramento del modello supervisionato
model = LogisticRegression(max_iter=1000)
model.fit(features_train, y_train)

# Predizione delle anomalie nei dati di test
y_pred = model.predict(features_test)
y_proba = model.decision_function(features_test)

# Visualizzazione dei risultati
print("Predizioni nel test set:", y_pred)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_proba)
print("Metriche di valutazione:\n", metrics)
# {'Accuracy': 0.977, 'Precision': 0.972, 'Recall': 0.92, 'F1': 0.945, 'MCC': 0.932, 'AUC_PR': 0.962, 'AUC_ROC': 0.984, 'PREC_N_SCORES': 0.929}

Predizioni nel test set: [0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 1
 0 0 1 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1
 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 0
 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0
 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


### Prova con Dettagli dal GitHub del Paper

In [ ]:
from sklearn.linear_model import Ridge

from rocket_functions import generate_kernels, apply_kernels

def detect_anomalies_with_threshold(scores, threshold):
    return (scores > threshold).astype(int)


# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 1000
kernels = generate_kernels(input_length, num_kernels)

# Applica i kernel alle serie temporali
features_train = apply_kernels(X_train2, kernels)
features_test = apply_kernels(X_test2, kernels)


# Addestramento del modello supervisionato
model = Ridge(alpha=1.0)
model.fit(features_train, y_train)

# Predizione delle anomalie nei dati di test
anomaly_scores_test = model.predict(features_test)
anomaly_scores_train = model.predict(features_train)

# Rilevamento delle anomalie
threshold = np.percentile(anomaly_scores_train , 95)
anomaly_labels_train = detect_anomalies_with_threshold(anomaly_scores_train , threshold)
anomaly_labels_test = detect_anomalies_with_threshold(anomaly_scores_test , threshold)

# Visualizzazione dei risultati
print("Predizioni nel test set:", anomaly_labels_test)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, anomaly_labels_test, y_proba=anomaly_scores_test)
print("Metriche di valutazione:\n", metrics)
#  {'Accuracy': 0.888, 'Precision': 0.966, 'Recall': 0.496, 'F1': 0.655, 'MCC': 0.644, 'AUC_PR': 0.922, 'AUC_ROC': 0.962, 'PREC_N_SCORES': 0.912}

Predizioni nel test set: [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0
 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


## LogisticClassifierCV ( SUPERVISED )

In [32]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeClassifierCV
from scipy.special import softmax

# ======================= ELABORAZIONE DATI TRAINING =============================
STEP = 250
# Lista per memorizzare i segmenti di training
X_train_final = []
y_train_final = []

# Leggi il file CSV
dfSegment = pd.read_csv("data/segments.csv", index_col="timestamp")
channelFix = "CADC0872"

# Itera su ogni segmento unico per il canale corrente
for segment in dfSegment[dfSegment["channel"] == channelFix]["segment"].unique():
    mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channelFix) & (dfSegment["segment"] == segment)

    # Filtra i dati in base alla maschera
    X_trainS = dfSegment.loc[mask, "value"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    y_trainS = dfSegment.loc[mask, "anomaly"] #.reset_index(drop=True).values  # Estrarre solo 'value'
    # print(X_trainS.shape)
    # Suddividi in sottoliste di STEP elementi
    for i in range(0, len(X_trainS) - STEP + 1, STEP):
        X_train_final.append(X_trainS[i:i + STEP])
        y_train_final.append(y_trainS[i])
        

# Converti la lista in un numpy array
X_train = np.array(X_train_final)
y_train = np.array(y_train_final)
# Reshape per ottenere la shape desiderata
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train = X_train.transpose(0, 2, 1)
# print(X_train_final.shape)


# ======================= ELABORAZIONE DATI TEST =============================

# Predisposizione del test set
test_data = dfSegment[dfSegment["train"] == 0]
# Predisposizione del test set
X_test_final = []
y_test_final = []

for segment in test_data[test_data["channel"] == channelFix]["segment"].unique():

    mask = (test_data["channel"] == channelFix) & (test_data["segment"] == segment)
    X_testS = test_data.loc[mask, "value"]#.reset_index(drop=True).values
    y_testS = test_data.loc[mask, "anomaly"]#.reset_index(drop=True).values
    
    for i in range(0, len(X_testS) - STEP + 1, STEP):
        X_test_final.append(X_testS[i:i + STEP])
        y_test_final.append(y_testS[i])


X_test = np.array(X_test_final)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = X_test.transpose(0, 2, 1)
# print("X_test: ",X_test)
# X_test = np.array(X_test_final).reshape(len(X_test_final), STEP, 1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


y_test = np.array(y_test_final)
print("y_test: ",y_test)

# ======================= PRE-PROCESSING =============================

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================

# Genera kernel convoluzionali casuali
input_length = X_train.shape[1]
num_kernels = 10000
rocket_transformer = Rocket(num_kernels = num_kernels, n_jobs=-1)

# Applica i kernel alle serie temporali
features_train = rocket_transformer.fit_transform(X_train)
features_test = rocket_transformer.transform(X_test)
print("features_train: ", features_train)

# Addestramento del modello supervisionato
model = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10))
model.fit(features_train, y_train)

# Predizione delle anomalie nei dati di test
y_pred = model.predict(features_test)

# Per separare multiclasse o monoclasse
if  len(np.unique(y_test)) > 2:
    y_proba = softmax(model.decision_function(features_test), axis=1)
else:
    y_proba = softmax(model.decision_function(features_test), axis=0)

# Visualizzazione dei risultati
print("Predizioni nel test set:", y_pred)

# Eseguiamo la valutazione delle metriche
metrics = evaluate_metrics(y_test, y_pred, y_proba)
print("Metriche di valutazione:\n", metrics)

C:\Users\franc\AppData\Local\Temp\ipykernel_8292\3872063633.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_final.append(y_trainS[i])
C:\Users\franc\AppData\Local\Temp\ipykernel_8292\3872063633.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


X_train shape: (53, 1, 250)
X_test shape: (15, 1, 250)
y_test:  [0 0 1 1 1 1 0 1 1 1 1 0 1 0 0]
features_train:      0         1      2         3         4         5      6          7      \
0   0.225  4.124670  0.952  6.125443  0.383117  7.832538  0.940  13.035572   
1   0.000 -2.807308  1.000  3.077709  0.811688  1.757264  0.984   2.071925   
2   0.675  1.213135  0.956  4.720337  0.675325  3.792974  0.960   4.592155   
3   0.975  2.809944  0.952  3.151166  0.383117  1.254861  0.960   3.584738   
4   0.400  0.753760  0.860  4.895823  0.623377  3.664900  0.948   9.445580   
5   0.100  0.217014  1.000  1.190639  0.435065  1.253239  0.996   3.147383   
6   0.975  2.094759  0.924  3.811384  0.214286  6.318000  0.964  11.139218   
7   0.275  0.742934  0.932  7.718774  0.402597  1.485129  0.952   4.986872   
8   1.000  2.478657  0.960  1.363665  0.012987  0.036954  0.996   3.232711   
9   0.000 -1.871216  1.000  2.061351  0.824675  1.793436  0.992   2.447036   
10  0.100  0.216995  1.000  1

In [31]:
print("features_train: ", features_train.shape)
print("y_train: ", y_train.shape)

features_train:  (53, 20000)
y_train:  (0,)


# Test Rocket su NASA

### ROCKET con NASA -> Unsupervised

In [2]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from NASA.nasa import NASA
from valutazione_metriche import evaluate_metrics
from rocket_functions import generate_kernels, apply_kernels

RANDOM_STATE = 42
STEP = 250
OUTPUT_FILE = "risultatiNASA_ROCKET.csv"

# ======================= ELABORAZIONE DATI TRAINING =============================

# Lista per memorizzare i segmenti di training
X_train_final = []

# Uso del dataset NASA per tutti i canali
dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly")
print(dataset.data.shape)
data = dataset.data
train = []
for i in range(0, data.shape[0] - STEP +1, STEP): 
    train.append(data[i:i+STEP])

train = np.stack(train)


# ======================= ELABORAZIONE DATI TEST =============================

dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly", train=False)
data = dataset.data
Test = []
output = []
o = np.zeros(data.shape[0])
for start,end in dataset.anomalies:
    o[start:end] = 1
for i in range(0, data.shape[0] - STEP +1, STEP): 
    Test.append(data[i:i+STEP])
    output.append(o[i:i+STEP])

output = np.stack(output)
Test = np.stack(Test)


# ======================= PRE-PROCESSING =============================

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
# X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================
# input_length = train.shape[0]
num_kernels = 10000

train = train.reshape(train.shape[0], -1)  # Da 3D a 2D
Test = Test.reshape(Test.shape[0], -1)
print("train: ", train.shape) 
print("TEST: ", Test.shape)

kernels = generate_kernels(STEP, num_kernels)

train = train.astype(np.float64)
features_train = apply_kernels(train, kernels)

Test = Test.astype(np.float64)
features_test = apply_kernels(Test, kernels)


# Applica i kernel alle serie temporali
features_train = apply_kernels(train, kernels)

features_test = apply_kernels(Test, kernels)



# RImozioni valori infiniti
features_train = np.nan_to_num(features_train, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)
features_test= np.nan_to_num(features_test, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)

# Addestramento del modello supervisionato
model = KNN()
model.fit(features_train)

# Predizione delle anomalie nei dati di test
y_pred = model.predict(features_test)
y_proba = model.decision_function(features_test)

threshold = np.percentile(y_proba, 95)  # Soglia al 95° percentile
predicted_anomalies = y_proba > threshold
print("Percentuale di anomalie rilevate:", predicted_anomalies.mean() * 100)

# Visualizzazione dei risultati
print("Predizioni nel test set:", y_pred)
print("Predizioni nel test set:", y_proba)

# Scegliere se una sequenda è un anomalia o no -> 10%
threshold = 25 # -> 10%
# Conta il numero di 1 in ogni lista
counts = np.sum(output, axis=1)
output = np.where(counts >= threshold, 1, 0)


print(output)
metrics = evaluate_metrics(output, predicted_anomalies)
print("Metriche di valutazione:\n", metrics)



(2648, 25)
train:  (10, 6250)
TEST:  (31, 6250)
Percentuale di anomalie rilevate: 6.451612903225806
Predizioni nel test set: [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Predizioni nel test set: [39.34577467 36.10637661 32.64133316 36.35686125 35.17089137 34.22729672
 34.3235441  40.1292137  41.66561789 33.69131247 32.59504588 34.15536612
 38.24332809 38.96896401 29.29555863 33.06146931 34.47508609 34.56070073
 34.81417879 33.43194699 35.46488308 34.60237567 37.23227165 34.80146113
 47.73908761 32.82958253 32.60580054 31.43605806 34.47250574 36.07161459
 30.1217598 ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.871, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': -0.069}


In [6]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from NASA.nasa import NASA
from valutazione_metriche import evaluate_metrics
from rocket_functions import generate_kernels, apply_kernels

RANDOM_STATE = 42
STEP = 250

results_df = pd.DataFrame(columns=["Channel", "Accuracy", "Precision", "Recall", "F1", "MCC", "AUC_ROC", "AUC_PR"])

# ======================= ELABORAZIONE DATI TRAINING =============================
# Itera su tutti i canali del dataset
for channel_id in NASA.channel_ids:
    print(f"Processing channel: {channel_id}")
    # Lista per memorizzare i segmenti di training
    X_train_final = []

    # Uso del dataset NASA per tutti i canali
    dataset = NASA("./datasets", channel_id, mode="anomaly")
    print(dataset.data.shape)
    data = dataset.data
    train = []
    for i in range(0, data.shape[0] - STEP +1, STEP): 
        train.append(data[i:i+STEP])

    train = np.stack(train)


    # ======================= ELABORAZIONE DATI TEST =============================

    dataset = NASA("./datasets", channel_id, mode="anomaly", train=False)
    data = dataset.data
    Test = []
    output = []
    o = np.zeros(data.shape[0])
    for start,end in dataset.anomalies:
        o[start:end] = 1
    for i in range(0, data.shape[0] - STEP +1, STEP): 
        Test.append(data[i:i+STEP])
        output.append(o[i:i+STEP])

    output = np.stack(output)
    Test = np.stack(Test)

    # ======================= FIT e PREDICT e SCORE =============================
    # input_length = train.shape[0]
    num_kernels = 10000

    train = train.reshape(train.shape[0], -1)  # Da 3D a 2D
    Test = Test.reshape(Test.shape[0], -1)
    print("train: ", train.shape) 
    print("TEST: ", Test.shape)

    kernels = generate_kernels(STEP, num_kernels)

    train = train.astype(np.float64)
    features_train = apply_kernels(train, kernels)

    Test = Test.astype(np.float64)
    features_test = apply_kernels(Test, kernels)


    # Applica i kernel alle serie temporali
    features_train = apply_kernels(train, kernels)

    features_test = apply_kernels(Test, kernels)



    # RImozioni valori infiniti
    features_train = np.nan_to_num(features_train, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)
    features_test= np.nan_to_num(features_test, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)

    # Addestramento del modello supervisionato
    model = KNN(n_neighbors=1)
    model.fit(features_train)

    # Predizione delle anomalie nei dati di test
    y_pred = model.predict(features_test)
    y_proba = model.decision_function(features_test)

    threshold = np.percentile(y_proba, 95)  # Soglia al 95° percentile
    predicted_anomalies = y_proba > threshold
    print("Percentuale di anomalie rilevate:", predicted_anomalies.mean() * 100)

    # Visualizzazione dei risultati
    print("Predizioni nel test set:", y_pred)
    print("Predizioni nel test set:", y_proba)

    # Scegliere se una sequenda è un anomalia o no -> 10%
    threshold = 25 # -> 10%
    # Conta il numero di 1 in ogni lista
    counts = np.sum(output, axis=1)
    output = np.where(counts >= threshold, 1, 0)


    print(output)
    metrics = evaluate_metrics(output, predicted_anomalies)
    print("Metriche di valutazione:\n", metrics)

    results_df = pd.concat([results_df, pd.DataFrame([{
            "Channel": channel_id,
            "Accuracy": metrics.get("Accuracy", 0),
            "Precision": metrics.get("Precision", 0),
            "Recall": metrics.get("Recall", 0),
            "MCC": metrics.get("MCC", 0),
            "AUC_PR": metrics.get("AUC_PR", 0),
            "AUC_ROC": metrics.get("AUC_ROC", 0),
            "F1": metrics.get("F1", 0),
        }])], ignore_index=True)

    print("=========================FINE CHANNEL=============================")
# ======================= SALVATAGGIO RISULTATI =============================
results_df.to_csv(OUTPUT_FILE, index=False)
print(f"Risultati salvati in {OUTPUT_FILE}")

Processing channel: A-1
(2880, 25)
train:  (11, 6250)
TEST:  (34, 6250)
Percentuale di anomalie rilevate: 5.88235294117647
Predizioni nel test set: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
Predizioni nel test set: [18.33543992 20.90955435 19.7170929  13.67316349 18.13719694 25.04026811
 20.08963078 17.04746814 16.69631453 14.23326539 16.49585339 25.02108608
 16.0548561  17.29007103 16.7440533  14.81597321 22.7248683  23.52491098
 22.18369744 18.75362211 17.98687596 18.99861365 22.21794342 18.2326525
 18.85961504 12.4573738  23.32769471 19.64676119 20.45612912 19.81123338
 30.4030754  19.74648274 16.15748844 21.77979617]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.912, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': -0.044}
=========================FINE CHANNEL=============================
Processing channel: A-2
(2648, 25)
train:  (10, 6250)
TEST:  (31, 6250)


C:\Users\franc\AppData\Local\Temp\ipykernel_8292\2033848077.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{


Percentuale di anomalie rilevate: 6.451612903225806
Predizioni nel test set: [0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0]
Predizioni nel test set: [28.89564656 33.4807405  23.03301614 25.15847188 30.80584259 27.88451496
 22.61984682 27.56667093 20.9344756  29.49424966 25.77373573 28.24021928
 24.10449011 21.45720891 21.79096132 27.84987167 21.98467674 29.31744725
 23.40769281 25.68252963 22.98914928 27.25459836 22.42709833 30.97936404
 31.32231227 22.73843068 29.14226221 27.66289435 23.58447464 26.16345155
 24.25772895]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.871, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': -0.069}
=========================FINE CHANNEL=============================
Processing channel: A-3
(2736, 25)
train:  (10, 6250)
TEST:  (32, 6250)
Percentuale di anomalie rilevate: 6.25
Predizioni nel test set: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Predizioni nel te

c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\CodiceTesi_Sync\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Percentuale di anomalie rilevate: 11.11111111111111
Predizioni nel test set: [0 0 0 0 0 0 0 0 0]
Predizioni nel test set: [23.0756684  30.05407848 34.90960003 32.2598797  24.70496345 32.09181994
 33.10435843 24.09130424 35.51510883]
[0 0 1 0 0 0 0 0 1]
Metriche di valutazione:
 {'Accuracy': 0.889, 'Precision': 1.0, 'Recall': 0.5, 'F1': 0.667, 'MCC': 0.661}
=========================FINE CHANNEL=============================
Processing channel: C-2
(764, 55)
train:  (3, 13750)
TEST:  (8, 13750)
Percentuale di anomalie rilevate: 12.5
Predizioni nel test set: [0 1 0 1 0 0 0 0]
Predizioni nel test set: [15.79729445 44.35559123 22.3510321  35.27945942 31.32262345 30.10263322
 31.98266952 24.28955607]
[0 1 0 0 0 0 1 0]
Metriche di valutazione:
 {'Accuracy': 0.875, 'Precision': 1.0, 'Recall': 0.5, 'F1': 0.667, 'MCC': 0.655}
=========================FINE CHANNEL=============================
Processing channel: D-1
(2849, 25)
train:  (11, 6250)
TEST:  (34, 6250)
Percentuale di anomalie rilevate: 

ValueError: Expected n_neighbors < n_samples_fit, but n_neighbors = 1, n_samples_fit = 1, n_samples = 1

#### Senza KNN

In [15]:
import numpy as np
import pandas as pd

from pyod.models.knn import KNN

from NASA.nasa import NASA
from valutazione_metriche import evaluate_metrics
from rocket_functions import generate_kernels, apply_kernels

RANDOM_STATE = 42
STEP = 250

# ======================= ELABORAZIONE DATI TRAINING =============================

# Lista per memorizzare i segmenti di training
X_train_final = []

# Uso del dataset NASA per tutti i canali
dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly")
print(dataset.data.shape)
data = dataset.data
train = []
for i in range(0, data.shape[0] - STEP +1, STEP): 
    train.append(data[i:i+STEP])

train = np.stack(train)


# ======================= ELABORAZIONE DATI TEST =============================

dataset = NASA("./datasets", NASA.channel_ids[1], mode="anomaly", train=False)
data = dataset.data
Test = []
output = []
o = np.zeros(data.shape[0])
for start,end in dataset.anomalies:
    o[start:end] = 1
for i in range(0, data.shape[0] - STEP +1, STEP): 
    Test.append(data[i:i+STEP])
    output.append(o[i:i+STEP])

output = np.stack(output)
Test = np.stack(Test)


# ======================= PRE-PROCESSING =============================

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
# X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================
# input_length = train.shape[0]
num_kernels = 10000

train = train.reshape(train.shape[0], -1)  # Da 3D a 2D
Test = Test.reshape(Test.shape[0], -1)
print("train: ", train.shape) 
print("TEST: ", Test.shape)

kernels = generate_kernels(STEP, num_kernels)

train = train.astype(np.float64)
features_train = apply_kernels(train, kernels)

Test = Test.astype(np.float64)
features_test = apply_kernels(Test, kernels)


# Applica i kernel alle serie temporali
features_train = apply_kernels(train, kernels)

features_test = apply_kernels(Test, kernels)



# RImozioni valori infiniti
features_train = np.nan_to_num(features_train, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)
features_test= np.nan_to_num(features_test, nan=0.0, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)

# Sintesi delle caratteristiche per esempio
anomaly_scores_train = np.mean(features_train, axis=1)  
anomaly_scores_test = np.mean(features_test, axis=1)  

# Rilevamento delle anomalie
threshold = np.percentile(anomaly_scores_train , 95)
anomaly_labels_train = detect_anomalies_with_threshold(anomaly_scores_train , threshold)
anomaly_labels_test = detect_anomalies_with_threshold(anomaly_scores_test , threshold)

# Scegliere se una sequenda è un anomalia o no -> 10%
threshold = 25 # -> 10%
# Conta il numero di 1 in ogni lista
counts = np.sum(output, axis=1)
output = np.where(counts >= threshold, 1, 0)


print(output)
metrics = evaluate_metrics(output, anomaly_labels_test)
print("Metriche di valutazione:\n", metrics)


(2648, 25)
train:  (10, 6250)
TEST:  (31, 6250)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Metriche di valutazione:
 {'Accuracy': 0.935, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'MCC': 0.0}


c:\Users\franc\OneDrive - University of Pisa\Università\Tesi\OPS-SAT-AD\Paper_OPS-SAT_Python\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Finale Prove ROCKAD -> ModelSelection

In [66]:
# ======================= ELABORAZIONE DATI TRAINING =============================
X_train_final = []
y_train_final = []

for channel in dfSegment["channel"].unique():
    for segment in dfSegment[dfSegment["channel"] == channel]["segment"].unique():
        mask = (dfSegment["train"] == 1) & (dfSegment["channel"] == channel) & (dfSegment["segment"] == segment)
        X_trainS = dfSegment.loc[mask, "value"]
        y_trainS = dfSegment.loc[mask, "anomaly"].reset_index(drop=True).values
        
        for i in range(0, len(X_trainS) - STEP + 1, STEP):
            X_train_final.append(X_trainS[i:i + STEP])
            y_train_final.append(y_trainS[i])

X_train = np.array(X_train_final).reshape(-1, STEP, 1).transpose(0, 2, 1)
y_train = np.array(y_train_final)

# ======================= ELABORAZIONE DATI TEST =============================
X_test_final = []
y_test_final = []

for channel in dfSegment["channel"].unique():
    for segment in test_data[test_data["channel"] == channel]["segment"].unique():
        mask = (test_data["channel"] == channel) & (test_data["segment"] == segment)
        X_testS = test_data.loc[mask, "value"]
        y_testS = test_data.loc[mask, "anomaly"]
        
        for i in range(0, len(X_testS) - STEP + 1, STEP):
            X_test_final.append(X_testS[i:i + STEP])
            y_test_final.append(y_testS[i])

X_test = np.array(X_test_final).reshape(-1, STEP, 1).transpose(0, 2, 1)
y_test = np.array(y_test_final)

# ======================= PRE-PROCESSING =============================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# ======================= FIT e PREDICT e SCORE =============================
rockad = ROCKAD(n_estimators=10, n_kernels=20000, n_jobs=-1, random_state=RANDOM_STATE, power_transform=False)
rockad.fit(X_train)

score_train = rockad.predict_proba(X_train).reshape(-1, 1)
score_test = rockad.predict_proba(X_test).reshape(-1, 1)

decision_func = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
decision_func.fit(score_train, y_train)

result = decision_func.predict(score_test)
result_binary = np.where(result == -1, 0, 1)

metrics = evaluate_metrics(y_test, result_binary, score_test)
print("Metriche di valutazione:\n", metrics)


C:\Users\franc\AppData\Local\Temp\ipykernel_26484\4218593192.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test_final.append(y_testS[i])


Metriche di valutazione:
 {'Accuracy': 0.477, 'Precision': 0.477, 'Recall': 1.0, 'F1': 0.646, 'MCC': 0.0, 'AUC_PR': 0.405, 'AUC_ROC': 0.299, 'PREC_N_SCORES': 0.29}
